In [1]:
!pip install datasets
!pip install evaluate
!pip install transformers


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoModelForCausalLM, pipeline
from datasets import Dataset
import evaluate
import numpy as np
from google.colab import drive
from tqdm import tqdm



In [3]:
drive.mount('/content/drive')

file_path_task1 = "/content/drive/MyDrive/Colab Notebooks/orientation-cz-train.tsv"
data_task1 = pd.read_csv(file_path_task1, sep="\t")

class_1 = data_task1[data_task1['label'] == 1]
class_0 = data_task1[data_task1['label'] == 0]

class_0_upsampled = resample(class_0, replace=True, n_samples=len(class_1), random_state=42)
balanced_data_task1 = pd.concat([class_1, class_0_upsampled])

train_data_task1, val_data_task1 = train_test_split(balanced_data_task1, test_size=0.1, stratify=balanced_data_task1['label'], random_state=42)

train_dataset_task1 = Dataset.from_pandas(train_data_task1)
val_dataset_task1 = Dataset.from_pandas(val_data_task1)

model_name = "distilbert-base-multilingual-cased"
tokenizer_task1 = AutoTokenizer.from_pretrained(model_name)
model_task1 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def preprocess_function_task1(examples):
    return tokenizer_task1(examples["text_en"], truncation=True, padding=True, max_length=128)

train_dataset_task1 = train_dataset_task1.map(preprocess_function_task1, batched=True)
val_dataset_task1 = val_dataset_task1.map(preprocess_function_task1, batched=True)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predicted_classes = np.argmax(predictions, axis=1)
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'], 3)
    return {"Accuracy": acc}
    output_dir="/content/drive/MyDrive/Colab Notebooks/task1res",

training_args_task1 = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/task1res",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_steps=500,
    fp16=True,
    load_best_model_at_end=True,

)


trainer_task1 = Trainer(
    model=model_task1,
    args=training_args_task1,
    train_dataset=train_dataset_task1,
    eval_dataset=val_dataset_task1,
    tokenizer=tokenizer_task1,
    compute_metrics=compute_metrics
)

trainer_task1.train()

predictions_task1 = trainer_task1.predict(val_dataset_task1)
preds_task1 = np.argmax(predictions_task1.predictions, axis=-1)
labels_task1 = val_dataset_task1['label']
print(classification_report(labels_task1, preds_task1))
print("Training over")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

Map:   0%|          | 0/4539 [00:00<?, ? examples/s]

Map:   0%|          | 0/505 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-5822b8e969f8>:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_task1 = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: laziratras (laziratras-metu-middle-east-technical-university). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.658000,0.600761,0.679000
2,0.535600,0.582243,0.764000
3,0.425600,0.773979,0.772000


              precision    recall  f1-score   support

           0       0.74      0.83      0.78       253
           1       0.80      0.70      0.75       252

    accuracy                           0.76       505
   macro avg       0.77      0.76      0.76       505
weighted avg       0.77      0.76      0.76       505

Training over
